In [ ]:
# Let's build our first MCP server!
# We'll create a simple greeting server with one tool

from typing import Any, Dict, List
import asyncio
from dataclasses import dataclass

# First, let's define our MCP server structure
# In a real MCP implementation, you'd import from the mcp library
# For this tutorial, we'll simulate the core concepts

@dataclass
class MCPTool:
    """Represents an MCP tool (function) that can be called"""
    name: str
    description: str
    parameters: Dict[str, Any]
    handler: callable

@dataclass
class MCPRequest:
    """Represents a request to an MCP tool"""
    tool_name: str
    arguments: Dict[str, Any]

@dataclass 
class MCPResponse:
    """Represents a response from an MCP tool"""
    success: bool
    result: Any = None
    error: str = None

class SimpleMCPServer:
    """A simplified MCP server for learning purposes"""
    
    def __init__(self, name: str):
        self.name = name
        self.tools: Dict[str, MCPTool] = {}
    
    def add_tool(self, tool: MCPTool):
        """Register a new tool with the server"""
        self.tools[tool.name] = tool
        print(f"✅ Registered tool: {tool.name}")
    
    def list_tools(self) -> List[str]:
        """Get list of available tools"""
        return list(self.tools.keys())
    
    async def call_tool(self, request: MCPRequest) -> MCPResponse:
        """Execute a tool and return the response"""
        if request.tool_name not in self.tools:
            return MCPResponse(
                success=False, 
                error=f"Tool '{request.tool_name}' not found"
            )
        
        try:
            tool = self.tools[request.tool_name]
            result = await tool.handler(**request.arguments)
            return MCPResponse(success=True, result=result)
        except Exception as e:
            return MCPResponse(
                success=False, 
                error=f"Tool execution failed: {str(e)}"
            )

# Create our first MCP server
server = SimpleMCPServer("HelloMCP Server")
print(f"🚀 Created MCP server: {server.name}")
print(f"📋 Available tools: {server.list_tools()}")


In [ ]:
# Now let's create our first tool - a greeting function!

async def greet_user(name: str, greeting_style: str = "friendly") -> str:
    """
    A simple greeting function that personalizes messages
    
    Args:
        name: The name of the person to greet
        greeting_style: Style of greeting (friendly, formal, excited)
    
    Returns:
        A personalized greeting message
    """
    
    greetings = {
        "friendly": f"Hello there, {name}! 👋 How are you doing today?",
        "formal": f"Good day, {name}. I hope you are well.",
        "excited": f"Hey {name}! 🎉 So great to meet you! This is awesome!",
        "casual": f"Hey {name}, what's up? 😎"
    }
    
    # Default to friendly if style not found
    greeting = greetings.get(greeting_style, greetings["friendly"])
    
    # Add a personal touch
    extra_msg = f"\\n\\n💡 By the way, your name '{name}' has {len(name)} letters!"
    
    return greeting + extra_msg

# Create the MCP tool definition
greeting_tool = MCPTool(
    name="greet_user",
    description="Generate a personalized greeting message for a user",
    parameters={
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the person to greet"
            },
            "greeting_style": {
                "type": "string", 
                "description": "Style of greeting",
                "enum": ["friendly", "formal", "excited", "casual"],
                "default": "friendly"
            }
        },
        "required": ["name"]
    },
    handler=greet_user
)

# Register the tool with our server
server.add_tool(greeting_tool)

print(f"📋 Server now has {len(server.tools)} tool(s): {server.list_tools()}")
print()
print("🎉 Your first MCP server is ready with a greeting tool!")


In [ ]:
# Let's test our MCP server! 
# We'll simulate requests from an AI assistant

async def test_mcp_server():
    """Test our MCP server with different requests"""
    
    print("🧪 Testing Your First MCP Server")
    print("=" * 50)
    print()
    
    # Test 1: Basic greeting
    print("📨 Test 1: Basic friendly greeting")
    request1 = MCPRequest(
        tool_name="greet_user",
        arguments={"name": "Alice"}
    )
    
    response1 = await server.call_tool(request1)
    if response1.success:
        print("✅ SUCCESS!")
        print(f"🤖 AI Response: {response1.result}")
    else:
        print(f"❌ ERROR: {response1.error}")
    
    print("\\n" + "-" * 30 + "\\n")
    
    # Test 2: Formal greeting
    print("📨 Test 2: Formal greeting style")
    request2 = MCPRequest(
        tool_name="greet_user", 
        arguments={"name": "Dr. Smith", "greeting_style": "formal"}
    )
    
    response2 = await server.call_tool(request2)
    if response2.success:
        print("✅ SUCCESS!")
        print(f"🤖 AI Response: {response2.result}")
    else:
        print(f"❌ ERROR: {response2.error}")
    
    print("\\n" + "-" * 30 + "\\n")
    
    # Test 3: Excited greeting
    print("📨 Test 3: Excited greeting style")
    request3 = MCPRequest(
        tool_name="greet_user",
        arguments={"name": "Bob", "greeting_style": "excited"}
    )
    
    response3 = await server.call_tool(request3)
    if response3.success:
        print("✅ SUCCESS!")
        print(f"🤖 AI Response: {response3.result}")
    else:
        print(f"❌ ERROR: {response3.error}")
    
    print("\\n" + "-" * 30 + "\\n")
    
    # Test 4: Error case - nonexistent tool
    print("📨 Test 4: Testing error handling (nonexistent tool)")
    request4 = MCPRequest(
        tool_name="calculate_meaning_of_life",
        arguments={}
    )
    
    response4 = await server.call_tool(request4)
    if response4.success:
        print("✅ SUCCESS!")
        print(f"🤖 AI Response: {response4.result}")
    else:
        print(f"❌ EXPECTED ERROR: {response4.error}")
    
    print("\\n🎉 All tests completed!")

# Run the tests
await test_mcp_server()
